# Problem 48 - Tree reconstruction

This problem was asked by Google.
Given pre-order and in-order traversals of a binary tree, write a function to reconstruct the tree.

For example, given the following preorder traversal:
[a, b, d, e, c, f, g]

And the following inorder traversal:
[d, b, e, a, f, c, g]

You should return the following tree:

```
    a
   / \
  b   c
 / \ / \
d  e f  g
```

## Solution

In [1]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
    def pre_order(self):
        yield self.value
        if self.left is not None: yield from self.left.pre_order()
        if self.right is not None: yield from self.right.pre_order()

    def in_order(self):
        if self.left is not None: yield from self.left.in_order()
        yield self.value
        if self.right is not None: yield from self.right.in_order()

In [2]:
tree = Node(
    'a',
    Node(
        'b',
        Node('d'),
        Node('e')
    ),
    Node(
        'c',
        Node('f'),
        Node('g')
    )
)

In [3]:
pre_order = list(tree.pre_order())
pre_order

['a', 'b', 'd', 'e', 'c', 'f', 'g']

In [4]:
in_order = list(tree.in_order())
in_order

['d', 'b', 'e', 'a', 'f', 'c', 'g']

In [5]:
# O(5*|pre_order|^2)
# Could be made linear if bounds for pre/in order lists 
# and hash map for finding the index of root in in_order list are used

def reconstruct(pre_order, in_order):
    if not pre_order or not in_order: return None

    root = pre_order[0]
    root_in_order_idx = in_order.index(root)

    return Node(
        root,
        reconstruct(pre_order[1:root_in_order_idx + 1], in_order[:root_in_order_idx]),
        reconstruct(pre_order[root_in_order_idx + 1:], in_order[root_in_order_idx + 1:])
    )

## Explanation

The first item in the pre order list is the root. All the items to the left of it are in the pre order list from 1 to the index of the root in the in order list. We slice the pre and in order lists based on this index and recurse. (Check for recursion boundary conditions).

Here ``r`` denotes the ``root`` and the ``arrows`` denote the way we ``slice`` the pre and in order arrays.

| #                 | 0 | 1 | 2 | 3 | 4 | 5 | 6 |
|-------------------|---|---|---|---|---|---|---|
| pre_prder slicing | r | > | . | < | > | . | < |
| pre_order         | a | b | d | e | c | f | g |
| in_order slicing  | > | . | < | r | > | . | < |
| in_order          | d | b | e | a | f | c | g |

## Tests

In [6]:
reconstruction = reconstruct(pre_order, in_order)

In [7]:
list(reconstruction.pre_order()) == pre_order

True

In [8]:
list(reconstruction.in_order()) == in_order

True

In [9]:
from random import random

def random_tree(tree_prob_size):
    value = 0

    def recurse(node_prob):
        nonlocal value
        value += 1
        return Node(value, recurse(node_prob / 2), recurse(node_prob / 2)) \
            if random() < node_prob \
            else None
        
    return Node('#', recurse(tree_prob_size), recurse(tree_prob_size))

In [10]:
rnd_tree = random_tree(100)
len(list(rnd_tree.in_order()))

673

In [11]:
from tqdm import tqdm

def stress_test():
    for _ in tqdm(range(10000)):
        actual_tree = random_tree(100)
        actial_pre_order = list(rnd_tree.pre_order())
        actual_in_order = list(rnd_tree.in_order())

        expected = reconstruct(pre_order, in_order)
        expected_pre_order = list(expected.pre_order())
        expected_in_order = list(expected.in_order())

        if expected_pre_order != pre_order or expected_in_order != in_order:
            print('Error')
            print('Actual', actial_pre_order, actual_in_order)
            print('Expected', expected_pre_order, expected_in_order)

In [12]:
stress_test()

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:13<00:00, 736.01it/s]
